In [ ]:
import streamlit as st
from streamlit_webrtc import webrtc_streamer, AudioProcessorBase, WebRtcMode
from streamlit_autorefresh import st_autorefresh
import numpy as np
import av
import io
import wave
import time
import queue
import speech_recognition as sr
from gtts import gTTS
import base64

In [ ]:
# Configuración de ICE (STUN y TURN)
rtc_configuration = {
    "iceServers": [
        {"urls": ["stun:stun.l.google.com:19302"]},
        {
            "urls": ["turn:relay1.expressturn.com:3478"],
            "username": "efBBNFJF1809NYOOA7",
            "credential": "mFYpag3yUL3bHv9j"
        }
    ]
}

In [ ]:
# Título y descripción
st.title("Conversación de Voz a Texto y Síntesis de Voz")
st.markdown("""
Esta aplicación captura tu voz, la convierte a texto y genera una respuesta automáticamente.  
El sistema mantiene el historial de la conversación y, una vez que se responde, vuelve a escuchar.
""")

In [ ]:
# Inicializar variables de sesión para el modo y el historial
if "conversation_history" not in st.session_state:
    st.session_state.conversation_history = []  # Lista de tuplas (emisor, mensaje)
if "mode" not in st.session_state:
    st.session_state.mode = "idle"  # Puede ser "idle", "listening" o "responding"

In [ ]:
# --- Definición de AudioProcessor ---
class AudioProcessor(AudioProcessorBase):
    def __init__(self):
        super().__init__()
        self.frames = []

    async def recv_queued(self) -> av.AudioFrame:
        frames = []
        while not self.queued_frames.empty():
            frame = await self.queued_frames.get()
            frames.append(frame)
        if frames:
            self.frames.extend(frames)
            # Retornamos el último frame para mantener la actualización interna
            return frames[-1]
        return None

In [ ]:
# --- Inicializar streamlit-webrtc (modo SENDONLY para capturar sin reproducir localmente) ---
webrtc_ctx = webrtc_streamer(
    key="speech-to-text",
    mode=WebRtcMode.SENDONLY,
    rtc_configuration=rtc_configuration,
    media_stream_constraints={"audio": True, "video": False},
    audio_processor_factory=AudioProcessor,
    audio_receiver_size=1024,
)

In [ ]:
# --- Función: convertir frames a WAV ---
def frames_to_wav(frames):
    try:
        all_samples = np.concatenate([frame.to_ndarray() for frame in frames], axis=1)
    except Exception as e:
        st.error(f"Error al concatenar frames: {e}")
        return None

    sample_rate = frames[0].sample_rate if frames else 48000
    try:
        n_channels = frames[0].layout.channels
        if not n_channels or n_channels < 1:
            raise ValueError("Número de canales no válido")
    except Exception as e:
        st.warning("No se pudo obtener el número de canales, usando 2 por defecto.")
        n_channels = 2

    wav_bytes_io = io.BytesIO()
    try:
        with wave.open(wav_bytes_io, "wb") as wf:
            wf.setnchannels(n_channels)
            wf.setsampwidth(2)  # 16 bits = 2 bytes
            wf.setframerate(sample_rate)
            if all_samples.dtype != np.int16:
                all_samples = all_samples.astype(np.int16)
            wf.writeframes(all_samples.tobytes())
        wav_bytes_io.seek(0)
        return wav_bytes_io
    except Exception as e:
        st.error(f"Error al crear el archivo WAV: {e}")
        return None

In [ ]:
# --- Función: convertir audio capturado a texto ---
def speech_to_text():
    # Esperamos unos segundos para acumular audio
    time.sleep(3)
    try:
        frames = webrtc_ctx.audio_receiver.get_frames(timeout=3)
    except queue.Empty:
        st.error("No se recibió audio en el tiempo esperado.")
        return None

    if not frames:
        st.error("No hay frames de audio para procesar.")
        return None

    wav_file = frames_to_wav(frames)
    if not wav_file:
        st.error("Error al crear el archivo de audio.")
        return None

    reconocedor = sr.Recognizer()
    try:
        with sr.AudioFile(wav_file) as source:
            audio_data = reconocedor.record(source)
        texto = reconocedor.recognize_google(audio_data, language="es-ES")
        return texto
    except sr.UnknownValueError:
        st.error("No se entendió lo que dijiste.")
        return None
    except sr.RequestError as e:
        st.error(f"Error en el servicio de reconocimiento: {e}")
        return None

In [ ]:
# --- Función: sintetiza respuesta con gTTS y retorna texto y audio en base64 ---
def responder_con_gTTS(texto):
    if "hola" in texto.lower():
        resp = "¡Hola! ¿Cómo puedo ayudarte?"
    elif "adiós" in texto.lower():
        resp = "Adiós, que tengas un buen día."
    else:
        resp = "No estoy seguro de cómo responder a eso."
    try:
        tts = gTTS(resp, lang="es")
        audio_bytes = io.BytesIO()
        tts.write_to_fp(audio_bytes)
        audio_bytes.seek(0)
        audio_base64 = base64.b64encode(audio_bytes.read()).decode("utf-8")
        return resp, audio_base64
    except Exception as e:
        st.error(f"Error al sintetizar la voz: {e}")
        return resp, None

In [ ]:
# --- Función: inyecta HTML para reproducir audio automáticamente ---
def reproducir_audio_autoplay(audio_base64):
    html_code = f"""
        <audio autoplay>
            <source src="data:audio/mp3;base64,{audio_base64}" type="audio/mp3">
            Tu navegador no soporta la reproducción automática.
        </audio>
    """
    st.markdown(html_code, unsafe_allow_html=True)

In [ ]:
# --- Modo conversacional ---
# Si el modo es "idle" mostramos un botón para iniciar la conversación
if st.session_state.mode == "idle":
    if st.button("Iniciar Conversación"):
        st.session_state.mode = "listening"
        st.experimental_rerun()

Si estamos en modo "listening" o "responding", activa autorefresh (cada 1 segundo)
if st.session_state.mode in ["listening", "responding"]:
   st_autorefresh(interval=1000, limit=0, key="conversation_refresh")

In [ ]:
# Lógica para el modo "listening": grabar y convertir a texto
if st.session_state.mode == "listening":
    st.info("Escuchando... Por favor, habla ahora.")
    texto_dicho = speech_to_text()
    if texto_dicho is not None:
        st.session_state.conversation_history.append(("Usuario", texto_dicho))
        st.session_state.mode = "responding"
        st.experimental_rerun()

In [ ]:
# Lógica para el modo "responding": generar respuesta y reproducir audio
if st.session_state.mode == "responding":
    st.info("Generando respuesta...")
    # Se usa el último mensaje del usuario para generar respuesta
    texto_ultimo = st.session_state.conversation_history[-1][1]
    respuesta_texto, audio_base64 = responder_con_gTTS(texto_ultimo)
    st.session_state.conversation_history.append(("Sistema", respuesta_texto))
    st.write(f"**Respuesta:** {respuesta_texto}")
    if audio_base64:
        reproducir_audio_autoplay(audio_base64)
    # Luego, una vez reproducida la respuesta, se vuelve a escuchar.
    # Ajusta el tiempo de espera según la duración del audio (aquí ponemos 5 segundos por ejemplo)
    time.sleep(5)
    st.session_state.mode = "listening"
    st.experimental_rerun()

In [ ]:
# Mostrar el historial de conversación
st.markdown("### Historial de la Conversación")
for emisor, mensaje in st.session_state.conversation_history:
    st.write(f"**{emisor}:** {mensaje}")

In [ ]:
# Botón para limpiar la conversación
if st.button("Limpiar Conversación"):
    st.session_state.conversation_history = []
    st.session_state.mode = "idle"
    st.experimental_rerun()